<a href="https://colab.research.google.com/github/Haritha-Kotte/python-langchain-weaviate/blob/main/meal_planner_weaviate_vectorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meal Planner

In the following RAG chain, we are doing the following steps:

1. Load Dataset from huggingface
2. Connect to weaviate
3. Create collection in weaviate with the properties
4. Ingest embedding data to weaviate collection
5. Create the retriever from vector store
6. Create a prompt with a template
7. Build the RAG chain
8. Invoke RAG chain and get the output
9. Close connection to weaviate

## Prerequisits

Before we proceed to do the above steps, we need to

- Create an account in weaviate and get the API key for the weaviate cluster
- Create an account in huggingface and get the API key for embedding generattion
- Create an account in openAI and get the API key for using LLM in the generation




In [1]:
!pip install weaviate-client datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/

## Load Dataset

In [3]:
from datasets import load_dataset

df = load_dataset("Shengtao/recipe")
recipes = df['train']
recipes = recipes.select(range(100))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

recipe.csv:   0%|          | 0.00/64.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32722 [00:00<?, ? examples/s]

## Connect to Weaviate

You have to enter the following credentials in google colab's secrets before connecting to weaviate

* WEAVIATE_CLUSTER
* WEAVIATE_KEY
* HUGGINGFACE_NEW_APIKEY

In [8]:
from google.colab import userdata
import weaviate
from weaviate.auth import Auth

cluster_url = userdata.get('WEAVIATE_CLUSTER')
auth_key = userdata.get('WEAVIATE_KEY')
huggingface_new_apikey = userdata.get('HUGGINGFACE_NEW_APIKEY')

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=cluster_url,
    auth_credentials=Auth.api_key(auth_key),
    headers={"X-HuggingFace-Api-Key": huggingface_new_apikey},
    skip_init_checks=True,
)

## Create collection in weaviate

In the following code block, we are creating a collection in weaviate with the properties from the recipe dataset

In [11]:
import weaviate.classes.config as wvcc

properties = [wvcc.Property(
                name=col,
                data_type=wvcc.DataType.TEXT
            ) for col in recipes.column_names if col != "embedding"]

weaviate_client.collections.delete("RecipeV4")
# Note that you can use `client.collections.create_from_dict()` to create a collection from a v3-client-style JSON object
collection = weaviate_client.collections.create(
    name="RecipeV4",
    description="A collection to store recipes",
    vectorizer_config=wvcc.Configure.Vectorizer.text2vec_huggingface(
        model="sentence-transformers/all-mpnet-base-v2",
        vectorize_collection_name=True
    ),
    properties=properties
)

## Ingest embedding data to Weaviate

In here, we are ingesting data with a rate limited batches as we are triggering huggingface API to generate embeddings as we upload the data and we don't want to hit the rate limit in huggingface.

After uploading the data, close the connection to weaviate.

In [12]:
try:
    with weaviate_client.batch.rate_limit(requests_per_minute=10) as batch:  # or <collection>.batch.rate_limit()

        for index, row in enumerate(recipes):
            data_object = {col: str(row[col]) for col in recipes.column_names}
            max_retries = 5
            for attempt in range(max_retries):
                try:
                    batch.add_object(properties=data_object, collection="RecipeV4") #Add object to the collection
                    print("Data uploaded successfully.")
                    break
                except weaviate.exceptions.UnexpectedStatusCodeException as e:
                    if '503' in str(e):
                        print(f"Attempt {attempt + 1}: Model is still loading, retrying...")
                        time.sleep(20)  # Wait and retry
                    if '429' in str(e):
                        # Handle rate limit error
                        retry_after = 60  # Retry-After header might be in seconds
                        print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
                        time.sleep(retry_after)
                    else:
                        raise  # Raise if it's a different error

finally:
    weaviate_client.close()

Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded successfully.
Data uploaded succes

Now, we have to install necessary packages for RAG pipeline.

In [16]:
!pip install langchain_weaviate langchain_huggingface langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.8 MB/s eta 0:00:00


## Create the Retriever from the vectorstore

Connect to the weaviate client and initialize the vectorstore with the embedding model we want to use for the query embedding.


In [15]:
from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

# Mention the embedding model name
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name
)

weaviate_client.connect()
# Initialise the vector store
vectorstore = WeaviateVectorStore(client=weaviate_client, index_name="RecipeV4", text_key="title", embedding=embeddings)
# Create the retriever to fetch relevant documents based on a query.
retriever = vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Create a prompt with the template

Created a prompt template that instructs the model to answer questions using retrieved context and to format as mentioned in it.

In [17]:
from langchain_core.prompts import ChatPromptTemplate

# Construct a template for the RAG mode
template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Show in a detailed information list format for the user to prepare the dishes and analyze the nutrition information of the dishes.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Show in a detailed information list format for the user to prepare the dishes and analyze the nutrition information of the dishes.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})]


## Build the RAG chain

We are now going to build a RAG chain that takes the retrieved result as the context and passes it to the prompt which later sends it to the LLM to generate the answer to the query.

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Connect to OpenAI GPT Model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=userdata.get("OPENAI_API_KEY"))
# Build the RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


### Recipe with Potatoes:

#### Basic Mashed Potatoes:
- **Ingredients:**
  - 2 pounds baking potatoes, peeled and quartered
  - 2 tablespoons butter
  - 1 cup milk
  - Salt and pepper to taste

- **Instructions:**
  1. Bring a pot of salted water to a boil.
  2. Add potatoes and cook until tender but still firm, about 15 minutes; drain.
  3. Heat butter and milk in a saucepan until butter is melted.
  4. Blend milk mixture into potatoes until smooth and creamy.
  5. Season with salt and pepper to taste.

- **Nutrition Information:**
  - Calories: 257.1
  - Fat: 7.2g
  - Saturated Fat: 4.5g
  - Cholesterol: 20.1mg
  - Carbohydrates: 43.7g
  - Protein: 5.6g
  - Fiber: 3.7g
  - Potassium: 763.1mg
  - Vitamin C: 15.2mg
  - Iron: 0.7mg
  - Calcium: 89.4mg

#### Roasted Potatoes and Onions:
- **Ingredients:**
  - 2 pounds potatoes, sliced into 1/2-inch-thick pieces
  - 1 onion, halved and each half cut into quarters
  - 1/2 cup canola oil
  - 1/2 cup olive oil
  - 4 cloves garlic
  - 1 enve

## Input the query

You can enter the query here

In [25]:
query = "Recipe with potatoes" # @param {"type":"string","placeholder":"Enter your query"}


## Invoke RAG chain

Invoke the RAG chain with the query and print the result

In [26]:
output = rag_chain.invoke(query)
print(output)

### Recipe with Potatoes:

#### Basic Mashed Potatoes:
- **Ingredients:**
  - 2 pounds baking potatoes, peeled and quartered
  - 2 tablespoons butter
  - 1 cup milk
  - Salt and pepper to taste
- **Instructions:**
  1. Bring a large pot of salted water to a boil. Add potatoes and garlic, lower heat to medium, and simmer until potatoes are tender, 15 to 20 minutes.
  2. Heat milk and butter in a small saucepan over low heat until butter is melted.
  3. Drain potatoes and return to the pot. Slowly add warm milk mixture, blending it in with a potato masher or electric mixer until potatoes are smooth and creamy. Season with salt and pepper.
- **Nutrition Information:**
  - Calories: 257.1
  - Fat: 7.2g
  - Carbohydrates: 43.7g
  - Protein: 5.6g
  - Fiber: 3.7g
  - Cholesterol: 20.1mg
  - Sodium: 76.1mg
  - Potassium: 763.1mg
  - Vitamin C: 15.2mg
  - Calcium: 89.4mg
  - Iron: 0.7mg

#### Roasted Potatoes and Onions:
- **Ingredients:**
  - 2 pounds potatoes, sliced into 1/2-inch-thick piece

## Close weaviate connection

In [27]:
weaviate_client.close()